<a href="https://colab.research.google.com/github/silverstar0727/pragmatism-AI/blob/main/VGG_cifar_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=3)

print("x shape")
print(f"train: {x_train.shape}, val: {x_val.shape}, test: {x_test.shape}")
print("y shape")
print(f"train: {y_train.shape}, val: {y_val.shape}, test: {y_test.shape}")


trainloader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
trainloader = trainloader.shuffle(1024).batch(128).prefetch(tf.data.experimental.AUTOTUNE)
valloader = tf.data.Dataset.from_tensor_slices((x_val, y_val))
valloader = valloader.batch(128).prefetch(tf.data.experimental.AUTOTUNE)
testloader = tf.data.Dataset.from_tensor_slices((x_test, y_test))
testloader = testloader.batch(128).prefetch(tf.data.experimental.AUTOTUNE)

x shape
train: (40000, 32, 32, 3), val: (10000, 32, 32, 3), test: (10000, 32, 32, 3)
y shape
train: (40000, 1), val: (10000, 1), test: (10000, 1)


In [ ]:
cfg = {
    'VGG11': ['M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

In [ ]:
class VGG(tf.keras.Model):
    def __init__(self, model_code):
        super(VGG, self).__init__()
        self.ls = self._make_layers(model_code)
        self.flatten = tf.keras.layers.Flatten()
        self.linear = tf.keras.layers.Dense(128, activation='relu')
        self.outputs = tf.keras.layers.Dense(10, activation='softmax')
        
    def call(self, x):
        for layer in self.ls:
            x = layer(x)
        x = self.flatten(x)
        x = self.linear(x)
        x = self.outputs(x)
        return x
    
    def _make_layers(self, model_code):
        layers = [tf.keras.layers.Conv2D(input_shape=(32, 32,3),
                                            filters=64,
                                            kernel_size=3,
                                            padding="same", 
                                            activation="relu")]
        for x in cfg[model_code]:
            if x != 'M':
                layers.append(tf.keras.layers.Conv2D(filters=x, 
                                       kernel_size=3,
                                       activation='relu', 
                                       padding='same'))
            else:
                layers.append(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))

        return layers

In [ ]:
net = VGG('VGG11')
#net = Model()
net.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

result = net.fit(
    trainloader, 
    epochs = 5,
    validation_data=valloader)

keras.backend.clear_session() 

Epoch 1/5
313/313 [==============================] - 8s 24ms/step - loss: 1.9387 - accuracy: 0.2566 - val_loss: 1.5245 - val_accuracy: 0.4222
Epoch 2/5
313/313 [==============================] - 7s 24ms/step - loss: 1.3799 - accuracy: 0.4906 - val_loss: 1.2060 - val_accuracy: 0.5568
Epoch 3/5
313/313 [==============================] - 7s 24ms/step - loss: 1.1203 - accuracy: 0.5928 - val_loss: 1.0777 - val_accuracy: 0.6187
Epoch 4/5
313/313 [==============================] - 7s 24ms/step - loss: 0.9442 - accuracy: 0.6607 - val_loss: 1.0002 - val_accuracy: 0.6560
Epoch 5/5
313/313 [==============================] - 7s 24ms/step - loss: 0.8233 - accuracy: 0.7081 - val_loss: 0.8791 - val_accuracy: 0.7004


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D , Flatten, Dropout
import numpy as np

def Model():
    model = Sequential()
    model.add(Conv2D(input_shape=(32, 32,3), filters=64,kernel_size=3, padding="same", activation="relu"))
    model.add(Conv2D(filters=64,kernel_size=3, padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=2, strides=2))
    model.add(Conv2D(filters=128, kernel_size=3, padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=2,strides=2))
    model.add(Conv2D(filters=256, kernel_size=3, padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=3, padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=2,strides=2))
    model.add(Conv2D(filters=512, kernel_size=3, padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=3, padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=2, strides=2))
    model.add(Conv2D(filters=512, kernel_size=3, padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=3, padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=2, strides=2))
    model.add(Flatten())
    model.add(Dense(units=512,activation="relu"))
    model.add(Dense(units=10, activation="softmax"))

    return model